# 3유형

## 문제 1
- 직원의 나이, 연봉, 근무 특정 등에 따른 이직 여부 관련 자료 로드 
- 로지스틱 회귀 분석을 수행

### 문제 1-1
- 로지스틱 회귀모델을 적합한 후, p-value가 0.05보다 작은 변수의 회귀 계수를 구하라
- 단 절편항은 제외한다. 

In [ ]:
import pandas as pd
# statsmodels를 사용한 이유는  sklearn에서는 p-value를 구할수 없다. 
from statsmodels.formula.api import logit

In [11]:
df = pd.read_csv("data/attrition.csv")
df.head()

,attrition,age,income,overtime
0,0,44,6467,1
1,0,41,6201,0
2,1,25,4017,1
3,0,28,6360,1
4,1,58,2369,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   attrition  6000 non-null   int64
 1   age        6000 non-null   int64
 2   income     6000 non-null   int64
 3   overtime   6000 non-null   int64
dtypes: int64(4)
memory usage: 187.6 KB


In [13]:
# attirition 데이터를 종속으로 age, income, ovetime은 독립 변수로 사용하는 모델을 생성
model = logit("attrition ~ age + income + overtime", data=df)
# 모델에 학습
result = model.fit()
# 회귀 분석 모델의 결과를 출력
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.387298
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              attrition   No. Observations:                 6000
Model:                          Logit   Df Residuals:                     5996
Method:                           MLE   Df Model:                            3
Date:                Fri, 28 Nov 2025   Pseudo R-squ.:                  0.4215
Time:                        16:00:30   Log-Likelihood:                -2323.8
converged:                       True   LL-Null:                       -4016.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.7724      0.246     31.650      0.000       7.291       8.254
age           -0.0044      0.

In [17]:
# p-value가 0.05보다 작은 피쳐는 income, Intercept(절편항)
# income의 회귀 계수 출력 
round(result.params['income'], 3)

np.float64(-0.002)

### 문제 1-2 
- 나이 데이터에서 1씩 증가할때 마다 이직의 확률을 (odds ratio)를 구하라
    - odds ratio는 e^회귀계수

In [18]:
import numpy as np

In [19]:
odds = np.exp(result.params['age'])

In [21]:
round(odds, 3)

np.float64(0.996)

### 문제 1-3
- 새로운 직원의 정보를 이용하여 이직 확률을 예측하라
    - 나이 40, 월급 4500, 야근 1

In [23]:
test = pd.DataFrame(
    {
        'age' : [40], 
        'income' : [4500], 
        'overtime' : [1]
    }
)

In [41]:
round(result.predict(test), 3)

0    0.697
dtype: float64

# 문제2
- 건물 특성과 난방 부하에 관한 자료 로드 
- 다중회귀분석을 수행

# 문제 2-1
- 모든 독립 변수를 포함하여 회귀 모델을 생성
- 절편을 제외한 모든 회귀 계수들의 합을 구하시오

In [26]:
import pandas as pd
from statsmodels.formula.api import ols
import numpy as np

In [27]:
df2 = pd.read_csv("data/heating.csv")
df2.head()

,heating_load,wall,roof,glazing,height
0,84.7,23.6,170.2,39.1,6.2
1,98.4,27.9,159.3,40.6,6.0
2,69.0,22.6,100.0,20.7,8.3
3,74.5,21.5,116.8,29.8,7.1
4,85.5,36.3,160.3,11.5,5.1


In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   heating_load  550 non-null    float64
 1   wall          550 non-null    float64
 2   roof          550 non-null    float64
 3   glazing       550 non-null    float64
 4   height        550 non-null    float64
dtypes: float64(5)
memory usage: 21.6 KB


In [30]:
model = ols("heating_load ~ wall + roof + glazing + height", data=df2).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           heating_load   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     417.8
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          2.02e-164
Time:                        16:25:56   Log-Likelihood:                -1772.0
No. Observations:                 550   AIC:                             3554.
Df Residuals:                     545   BIC:                             3576.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.3821      1.504     25.517      0.0

In [33]:
round(model.params[1:].sum(), 3)

np.float64(0.254)

### 문제 2-2
- 유의한 변수(p-value의 값이 0.05보다 작은 변수)를 사용하여 다중회귀분석을 다시 수행
- 결정계수(r2)값을 구하라

In [36]:
# 위의 summary에서 p-value가 0.05 이하인 피쳐는 roof, glazing, height 컬럼 3개 
model2 = ols('heating_load ~ roof + glazing + height', data=df2).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:           heating_load   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     556.7
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          1.31e-165
Time:                        16:29:33   Log-Likelihood:                -1772.5
No. Observations:                 550   AIC:                             3553.
Df Residuals:                     546   BIC:                             3570.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.2337      1.247     31.467      0.0

### 문제 2-3
- 새로운 건물에 대한 난방 부하 예측값을 출력
    - 벽 면적 20, 지붕면적 150, 유리 면적 20, 건물 높이 5

In [37]:
test2 = pd.DataFrame(
    {
        'wall' : [20], 
        'roof' : [150], 
        'glazing' : [20], 
        'height' : [5]
    }
)

In [40]:
round(model2.predict(test2), 3)[0]

np.float64(79.612)